## Korzystanie z API Open AI z użyciem _structured outputs_

**Structured outputs** to funkcjonalność LLM, która pozwala na otrzymywanie odpowiedzi w ściśle określonej strukturze. Zamiast otrzymywać tekst w formie swobodnej, możemy zdefiniować dokładny format odpowiedzi (np. JSON), co ułatwia dalsze przetwarzanie danych przez aplikację.

**Główne zalety:**
- Przewidywalny format odpowiedzi
- Łatwiejsze przetwarzanie przez aplikację
- Mniejsze ryzyko błędów przy parsowaniu odpowiedzi
- Możliwość wymuszenia konkretnej struktury danych

[Dokumentacja OpenAI - Structured Outputs](https://platform.openai.com/docs/guides/structured-outputs)

### Generowanie informacji w postacji JSON

Przykład w którym jako oczekiwaną strukturę odpowiedzi podajemy klasę Pydantica

In [ ]:
from openai import OpenAI
import os
from pydantic import BaseModel

In [ ]:
# tworzymy klasę pydantica opisującą strukturę informacji do zwrócenia
# w tym przypadku jest to postać w grze RPG
class RPGCharacter(BaseModel):
    genre: str
    name: str
    race: str
    class_type: str
    skills: list[str]

In [ ]:
api_key = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key=api_key)

response = client.beta.chat.completions.parse( # uwaga tym razem 'parse' zamiast 'create'
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "Jesteś ekspertem od gier RPG. Zawsze tworzysz postacie w języku polskim."},
        #{"role": "user", "content": "Przygotuj postać do gry RPG."},
        {"role": "user", "content": "Przygotuj postać do gry RPG Warhammer 40k."},
    ],
    response_format = RPGCharacter,
    temperature=0.7,
    max_tokens=1000
)

res = response.choices[0].message.parsed # pobieramy wynik z odpowiedzi, od razu sprasowany do obiektu

In [ ]:
json_data = res.model_dump_json(indent=4)
print(json_data)

### Generowanie informacji w postacji JSON (tym razem podając JSON Schema)

Przykład w którym jako oczekiwaną strukturę odpowiedzi podajemy bezpośrednio schemę JSONa

In [ ]:
from openai import OpenAI
import os
import json

In [ ]:
#tym razem posłużymy się schematem JSON Schema (wczytamy go z pliku)
with open('004. RPGCharacter.schema', 'r') as file:
    json_schema = file.read()

json_schema = json.loads(json_schema)

#print(json.dumps(json_schema, indent=4))

In [ ]:
api_key = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key=api_key)

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "Jesteś ekspertem od gier RPG. Zawsze tworzysz postacie w języku polskim."},
        {"role": "user", "content": "Przygotuj postać do gry RPG."},
    ],
    response_format={
        "type": "json_schema", 
        "json_schema": json_schema
    },
    temperature=0.7,
    max_tokens=1000
)

In [ ]:
# podejrzyjmy, co dostaliśmy
json_string = response.choices[0].message.content
formatted_json = json.dumps(json.loads(json_string), indent=4, ensure_ascii=False)
print(formatted_json)

### Generowanie informacji w postacji JSON (bardziej szczegółowo opisanego)

Ponownie operaujemy klasami pydantica, tym razem robimy to w sposób bardziej zaawansowany:
- dodajemy opisy do każdego pola
- mamy strukturę wielopoziomową - część pól klasy `RPGCharacter` jest innymi obiektami 

In [ ]:
from openai import OpenAI
import os
from pydantic import BaseModel, Field

In [ ]:
# tworzymy sobie klasę pydantica opisującą strukturę informacji do zwrócenia
# w tym przypadku jest to postać w grze RPG
# tym razem dodatkowo tworzyny osobne klasy na atrybuty postaci oraz informacje o umiejętnościach
class CharacterAttributes(BaseModel):
    strength: int = Field(description="Siła postaci") # w ten sposób dodajemy opis do pola
    dexterity: int = Field(description="Zręczność postaci")
    constitution: int = Field(description="Wytrzymałość postaci")
    intelligence: int = Field(description="Inteligencja postaci")
    wisdom: int = Field(description="Mądrość postaci")
    charisma: int = Field(description="Charyzma postaci")

class Skill(BaseModel):
    name: str = Field(description="Nazwa umiejętności")
    description: str = Field(description="Opis umiejętności")

class RPGCharacter(BaseModel):
    genre: str = Field(description="Gatunek gry RPG (np. fantasy, sci-fi)")
    name: str = Field(description="Imię postaci")
    race: str = Field(description="Rasa postaci (np. człowiek, elf, krasnolud)")
    class_type: str = Field(description="Klasa lub profesja postaci (np. wojownik, mag)")
    skills: list[Skill] = Field(description="Lista umiejętności specjalnych postaci")
    attributes: CharacterAttributes = Field(description="Atrybuty postaci")

In [ ]:
api_key = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key=api_key)

response = client.beta.chat.completions.parse( # uwaga tym razem 'parse' zamiast 'create'
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "Jesteś ekspertem od gier RPG. Zawsze tworzysz postacie w języku polskim."},
        #{"role": "user", "content": "Przygotuj postać do gry RPG."},
        #{"role": "user", "content": "Przygotuj postać do D&D w świecie Planescape."},
        #{"role": "user", "content": "Przygotuj postać do gry RPG Warhammer 40k. To ma być prawdziwy brutal"},
        #{"role": "user", "content": "Przygotuj postać do gry post-apo. Chcę postać posługującą się psioniką"},
        {"role": "user", "content": "Przygotuj postać do gry RPG w świecie Gwiezdnych Wojen. Chcę postać posługującą się zarówno jasną jak i ciemną stroną Mocy"},
        #{"role": "user", "content": "Przygotuj postać do gry RPG w świecie łączącym elementy fantasy i sci-fi. Chcę postać posługującą się zarówno magiczną technologią"},
    ],
    response_format = RPGCharacter,
    temperature=0.7,
    max_tokens=1000
)

res = response.choices[0].message.parsed

In [ ]:
json_data = res.model_dump_json(indent=4)
print(json_data)

### Ekstrakcja informacji z tekstu

A teraz przykład z ekstrakcją informacji o postaciach historycznych (z Wikipedii) lub fantastycznych (z Fandomu) z tekstów o nich mówiących

In [ ]:
from openai import OpenAI
import os
import wikipediaapi
from pydantic import BaseModel, Field
import fandom

In [ ]:
# tworzymy sobie klasę pydantica opisującą strukturę informacji do zwrócenia
# w tym przypadku jest to postać historyczna
class Education(BaseModel):
    institution: str = Field(description="Nazwa instytucji edukacyjnej")
    field_of_study: str = Field(description="Obszar wykształcenia")
    degree: str = Field(description="Tytuł lub stopień którego dotyczy wykształcenie")
    graduated: bool = Field(description="Czy postać ukończyła studia")

class Figure(BaseModel):
    name: str = Field(description="Pełne imię i nazwisko postaci historycznej")
    birth_date: str = Field(description="Data urodzenia")
    death_date: str = Field(description="Data śmierci. Puste jeśli postać żyje")
    places_of_residence: list[str] = Field(description="Lista miejsc zamieszkania")
    birth_place: str = Field(description="Miejsce urodzenia")
    death_place: str = Field(description="Miejsce śmierci. Puste jeśli postać żyje")
    occupation: list[str] = Field(description="Lista zawodów i funkcji")
    major_achievements: list[str] = Field(description="Lista najważniejszych osiągnięć")
    education: list[Education] = Field(description="Wykształcenie i miejsca studiów")
    nationality: list[str] = Field(description="Narodowość lub narodowości postaci")
    historical_period: str = Field(description="Okres historyczny, w którym żyła postać")

Uruchom jedną z dwóch poniższych komórek - zależnie czy chcesz pobierać informacje z Wikipedii czy z Fandomu

In [ ]:
# pobieramy informacjie z wikipedii
wiki = wikipediaapi.Wikipedia(language='pl', user_agent='Python Script')
page = wiki.page("Mikołaj_Kopernik")
#page = wiki.page("Aleksander_Kwaśniewski")
#page = wiki.page("Gandalf")

text = page.text

#print(text)

In [ ]:
# pobieramy informacjie z Fandomu (dawniej Wikia)
fandom.set_wiki("starwars")
page = fandom.page("Luke_Skywalker")

#fandom.set_wiki("lotr")
#page = fandom.page("Gandalf")
#page = fandom.page("Aragorn_II")

text = f"""
Title: {page.content['title']}
---
Summary:
{page.content['content']}
---
Infobox:
{page.content['infobox']}
"""

#print(text)

Prompt, którym będziemy wyciągać informacje

In [ ]:
# przygotowanie prompta z tekstem do przetworzenia
prompt = f""" Wyekstrahuj informacje o omawianej postaci z poniszego tekstu. Jeśli jest w innym języku niż polski, przetłumacz go na polski.
---
{text}
"""

... i strzał do modelu

In [ ]:
api_key = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key=api_key)

response = client.beta.chat.completions.parse( # uwaga tym razem 'parse' zamiast 'create'
    model="gpt-4o",
    messages=[
        {"role": "system", "content": """
            Jesteś ekspertem od ekstrakcji informacji z tekstów.
            Opierasz się tylko na dostarczonych danych.
            Jeśli czegoś nie ma w tekście, odpowiadające pole zostawiasz puste.
            Jeśli tekst źródłowy jest w innym języku, tłumacz wyekstrahowane informacje na polski.
            """},
        {"role": "user", "content": prompt},
    ],
    response_format = Figure,
    temperature=0, # nie chemy aby model wykazał się kreatywnością
    max_tokens=16000
)

res = response.choices[0].message.parsed

In [ ]:
json_data = res.model_dump_json(indent=4)
print(json_data)